In [28]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("WDC-DataFrame").getOrCreate()
#créer une session dans le master
spark = SparkSession.builder.master("spark://172.20.53.96:7077").appName("WDC-dico").getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-dico").getOrCreate()


#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = '***REMOVED***'
aws_secret_access_key = '***REMOVED***'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql import functions as f

from pyspark.sql import Row
import re
from urllib.parse import urlparse

df = spark.read.text("s3a://wdc/")

#Split the line into four parts using the ' ' separator
split_col = F.split(F.col("value"), "\s+", 3)

#Create new columns for the subject, predicate, object, and graph
df = df.withColumn("subject", split_col[0])
df = df.withColumn("predicate", split_col[1])
df = df.drop("value")

# permet de ne plus différencier les predicats http, https, www...
df = df.withColumn("predicate", f.regexp_replace(f.col("predicate"), "([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?", ""))
df.show(truncate=0)

df.createOrReplaceTempView("Super")
#spark.sql("select count(*) as count from Super").show()
 

In [ ]:
# Trie par le nombre d'occurence de chaque prédicat
preds=spark.sql("select predicate, count(*) as nb from Super group by predicate order by nb DESC")
preds.persist()
preds.show(truncate=0)

# Utilise le compte pour attribuer des ids croissantes
preds_final = preds.withColumn("id", F.monotonically_increasing_id())  
preds_final = preds_final.drop("nb")
preds_final.show(truncate=0)

# Sauvegarde le résultat
preds_final.write.option("header",True) \
  .mode("overwrite") \
  .csv("s3a://test-out/dico_reel_total")
